## Overview
This notebook creates the following matrices/files
- $D$ the file `epa.txt`
- $P^T$ the file `multixcan_res_10027.txt`
- $Y$ the file 

## Data needed for this notebook to work
- [EPA toxcast data](https://www.epa.gov/chemical-research/exploring-toxcast-data#Download) (this uses version 3.3)
- [PhenomeXcan data](https://github.com/hakyimlab/phenomexcan) 
- [SIDER side effects or indications](http://sideeffects.embl.de/download/)

Put these into the "input_data" directory

## FIrst, work on getting CUIs for each UKBiobank phenotype

In [4]:
import pandas as pd 

phenomeXcan = 'input_data/'
spredi = pd.read_csv(phenomeXcan + "s-predixcan-phenomexcan.csv",sep=",")
spredi = spredi.loc[~spredi['full_code'].str.startswith(tuple("20114|20112|20113|20107|20110|2976|20111".split("|"))) &
                   ~spredi['full_code'].str.lower().str.contains("medication")]
spredi = spredi.loc[spredi['n_cases'] > 200,:]

toget = []
icds = []
ncis = []
cancercode = []
for i in spredi['description']:
    if i.startswith("Non-cancer"):
        nci = i.split(":")[1].strip()
        toget.append(nci)
        ncis.append(nci)
    elif i.startswith("Diagnoses - main ICD10"):
        code = i.split(":")[1].split()[0].strip()
        toget.append(code)
        icds.append(code)
    elif i.startswith("Cancer code"):
        code = i.split(":")[1].strip()
        toget.append(code)
        cancercode.append(code)
    else:
        toget.append(i)
        

In [6]:
spredi['description'].str.lower().str.contains('leukemi').sum()

0

In [2]:
## use UMLS table to look up CUI for coded phenotypes
icd10 = pd.read_csv("input_data/icd10",sep="|",usecols=[0,13],header=None)
icdcui = {i:list(set(icd10.loc[icd10[13]==i,0].values)) for i in icds}

## for non-coded phenotypes, use API to get CUIs
left = set(toget)- set(icds) #- set(ncis) - set(cancercode)
left = [l for l in left if not l.startswith("Job ")]
with open("missing_phe.txt",'w') as f:
    f.write("\n".join(left) + "\n")

Use the script `umls_search_cui.sh` to get the matching possible CUIs. `bash umls_search_cui.sh missing_phe.txt` (requires UMLS API scripts to be available locally).

Once run, read in the results:

In [5]:
def read_res(fn):
    illcui = {}
    for i in open(fn):
        li = i.strip().split("%%%")
        cuis = li[1].strip(",").split(",") #[j.strip(",") for j in li if j[0]=="C" and j[1:].strip(",").isdigit()]
        illcui[li[0]] = cuis
        #cuis.append(name.split()[-1])
        #li = i.strip().replace(", C",",C").replace(", N",",N").split()
        #illcui[" ".join(name.split()[:-1])] = cuis #li[-1].split(",")
    return illcui
nonicd = read_res("intermediate_files/resmissing_phe.txt")

Now combine the ICD matches with the searched matches to get our cuis for each phenotype. This is not perfect but we will look at how it lines up with the SIDER phenotypes.

In [6]:
cuis = {k:icdcui[k] if k in icdcui else
       nonicd[k] if k in nonicd else
        [] if k.startswith("Job ") else 'WHAT' for k in toget}

gwascuis = {dd:cuis[toget[i]] for i, dd in enumerate(spredi["description"])}


## Now, load SIDER to match phenotypes using CUIs

Load in SIDER and get Meddra terms and UMLS CUIs

In [7]:
## this file is obtained directly from SIDER
sider = pd.read_csv("input_data/meddra_all_se.tsv.gz",sep="\t",header=None)
sider['cid'] = sider[0].str.slice(4).map(int)

sider.columns = ['stitch_flat','stitch_stereo','UMLS_label', 'type','meddra','se_name','pubchem_cid']
pt = sider.loc[sider['type']=='PT',].drop_duplicates(['stitch_flat','meddra']).copy()

from collections import Counter
pt_ae = pd.DataFrame(Counter(pt['meddra']),index=['ct']).transpose()
ptfreq = pt.loc[pt['meddra'].isin(pt_ae.loc[(pt_ae['ct']>4),:].index),:]

Now we make a file that uses the cuis to link MedDra terms to Phenotypes. 

In [8]:
all_se = ptfreq.loc[:,['meddra','se_name']].drop_duplicates('meddra')
ukmatch = {}
matchnames = []
manual = {}
for i in all_se['meddra']:
    mat = [k for k,v in gwascuis.items() if i in v]
    ukmatch[i] = mat
    matchnames.append('"' + '",'.join(mat) + '"')
    if len(mat):
        manual[all_se.loc[all_se['meddra']==i,'se_name'].values[0]] = mat

In [15]:
manual

{'Anaemia': ['Non-cancer illness code, self-reported: anaemia'],
 'Arrhythmia': ['Non-cancer illness code, self-reported: heart arrhythmia',
  'Non-cancer illness code, self-reported: irregular heart beat'],
 'Atrial fibrillation': ['Non-cancer illness code, self-reported: atrial fibrillation'],
 'Back pain': ['Non-cancer illness code, self-reported: back pain',
  'Dorsalgia',
  'Diagnoses - main ICD10: M54 Dorsalgia'],
 'Bronchitis': ['Non-cancer illness code, self-reported: bronchitis',
  'Bronchitis'],
 'Chest pain': ['Chest pain or discomfort'],
 'Constipation': ['Non-cancer illness code, self-reported: constipation'],
 'Cough': ['Diagnoses - main ICD10: R05 Cough'],
 'Depression': ['Non-cancer illness code, self-reported: depression',
  'Depression'],
 'Dysgeusia': ['Non-cancer illness code, self-reported: gout'],
 'Dyspepsia': ['Non-cancer illness code, self-reported: dyspepsia / indigestion',
  'Diagnoses - main ICD10: K30 Dyspepsia'],
 'Rash': ['Diagnoses - main ICD10: R21 Rash

Then, a student removed some spurious or too-general linkages (ie, "Neoplasms"), saved in the file `manual_matching_curation.py`

This is a dictionary where the keys are MedDra terms in SIDER and the values are Phenotypes in UKBiobank/PhenomeXcan

In [10]:
sys.path.append("./code")
import manual_matching_curation
manual = manual_matching_curation.get_manual()

In [11]:
sider2ukb = {}
for k in manual:
    x = spredi.loc[spredi['description'].isin(manual[k]),:].sort_values('n_cases',ascending=False).iloc[0,:]
    sider2ukb[k] = [x['description'], x['n_cases']]



from collections import defaultdict
grouped = defaultdict(list)
for k,v in sider2ukb.items():
    grouped[v[0]].append(k)
    


sidername2ukbname = {}
for k, v in grouped.items():
    for sname in v:
        sidername2ukbname[sname] = k

pt_match =  ptfreq.loc[ptfreq['se_name'].isin(sidername2ukbname),:].copy()
pt_match['ukb'] = [sidername2ukbname[n] for n in pt_match['se_name']]

pt_match['ae'] = 1
ptstack = pt_match.loc[:,['pubchem_cid','ukb','ae']].drop_duplicates(['pubchem_cid','ukb']).set_index(['pubchem_cid','ukb']).sort_index() #.unstack().shape
ptstack = ptstack.transpose().stack('ukb')
ptstack = ptstack.mask(pd.isnull(ptstack), other=0)
ptstack = ptstack.droplevel(0,0)

In [7]:
ptstack.shape

(197, 1402)

In [49]:
ptstack.to_csv("data_for_regression/sider_y.txt",sep="\t")

In [7]:
ptstack.shape

(197, 1402)

Now do it for indications 
- load in and clean up 
- use the same dictionary to get the unified names of phenotypes
- save as binary matrix

In [14]:
sider = pd.read_csv("input_data/meddra_all_indications.tsv.gz",sep="\t",header=None)
sider['cid'] = sider[0].str.slice(4).map(int)
sider.columns = ['stitch_flat','stitch_stereo','source','UMLS_label', 'type','meddra','se_name','pubchem_cid']
pt = sider.loc[sider['type']=='PT',].drop_duplicates(['pubchem_cid','meddra']).copy()

pt['ae'] = 1
ptstack = pt.loc[:,['pubchem_cid','se_name','ae']].drop_duplicates(['pubchem_cid','se_name']).set_index(['pubchem_cid','se_name']).sort_index() #.unstack().shape
ptstack = ptstack.transpose().stack('se_name')
ptstack = ptstack.mask(pd.isnull(ptstack), other=0)
ptstack.index = ptstack.index.droplevel(0)

ptren = ptstack.loc[ptstack.index.isin(sidername2ukbname.keys()),:].rename(sidername2ukbname)

ptren = ptren.loc[:,(ptren.sum(axis=0) > 0)]

ptren.to_csv("data_for_regression/sider_indications_y.txt",sep="\t")

Completed side effect/indication data matched to PhenomeXcan.

## Processing ToxCast data
First, create list of all compounts SMILES and use [PubChem](https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi) to convert these to Pubchem CID

In [70]:

epa = pd.read_csv("input_data/INVITRODBv3_20181017.csv",sep=",")

y = epa.loc[epa['SMILES']!="-",'SMILES']
z = [i.split()[0] if len(i.split()) > 1 else i for i in y ]

### make file for pubchem upload
with open("intermediate_files/smiles_clean",'w') as f:
    f.write("\n".join(z) + "\n")


Load in the result saved in `smiles_invitrodb`

In [71]:
  
smiles2cid = pd.read_csv("intermediate_files/smiles_invitrodb",sep="\t")
smiles2cid.columns = ['SMILES','cid']
smiles2cid = smiles2cid.drop_duplicates()

epa2 = epa.merge(smiles2cid,on='SMILES',how='left')
epa2['casrn-clean'] = epa2['CASRN'].str.replace("-","").str.replace("_","")

Now get the sider pubchem IDs, we will just keep these (there are many others in toxcast)

In [72]:
subsel = epa2.loc[epa2['cid'].isin(sider['pubchem_cid']),'casrn-clean'].values

In [33]:
len(subsel)

430

Go through the various databases and extract the level 5 data, particularly "endpoint" (`aenm`) and the drug tested (`code`), and the results `hit_pct`.

In [ ]:
dbs = {}
import glob
for i in glob.glob("INVITRODB_V3_3_SUMMARY/EXPOR*csv"):
    name = os.path.basename(i).replace("EXPORT_LVL5&6_","").replace("_200730.csv","")
    dbs[name] = []
    for ep in pd.read_csv(i,sep=",", chunksize=50000):
        code = ep['code'].mask(pd.isnull(ep['code']),other='').str.slice(1)
        ep['code-cln']= code    
        print(' chems=',
             len(set(code)),  "/", 
             len(set(code) - set(epa2['casrn-clean'])), 
              ' endpoints=', len(set(ep['aenm'].mask(pd.isnull(ep['aenm']),other=''))))
        dbs[name].append(ep.loc[ep['code-cln'].isin(subsel),])
    dbs[name]= pd.concat(dbs[name],axis=0)
    
x = pd.concat([v.loc[:,['code-cln','aenm',u'cnst_prob',  u'hill_prob', u'gnls_prob',
                    u'hit_pct', u'total_hitc', 
       u'cnst_pct', u'hill_pct', u'gnls_pct']].groupby(['code-cln','aenm']).agg('mean').reset_index()
           for v in dbs.values()],axis=0)
x['cid'] = epa2.set_index("casrn-clean").loc[list(x['code-cln'].values),'cid'].values

xstk = x.loc[:,['cid','aenm','hit_pct']].set_index(['cid','aenm']).sort_index().transpose().stack('aenm')
xstk = xstk.droplevel(0,0)
xstk.to_csv("epa.txt")

Now that we have the EPA we can filter the SIDER compounds to the ones we want to use

In [8]:
xstk = pd.read_csv("data_for_regression/epa.txt",sep="\t",index_col=0)

Since this matrix has many missing values, we use the SoftImpute method to reduce dimensionality and avoid missing values. This gives the factorization we refer to as $D = U_{D}S_DV_D^T$.

Because SoftImpute has some hyperparameters (rank and lambda), we set some fraction of these to be missing; then impute these; in order to select the parameters for the rank  and lambda. Finally, we save these matrices, where $U_DS_D$ has 429 rows (drugs) and 246 columns (rank selected) with lambda 0.3393222.

The R code to perform this analysis is below.

```
require(softImpute)

epa  = read.csv('epa.txt', sep="\", header=T)

epa2 = epa
epa2[sample(1:596739,37296,replace=FALSE)]=NaN

lam_epa = lambda0(epa2)
lambda_values_epa=exp(seq(from=log(10),to=log(0.1),length=50))
frobenius_epa  <- rep(NA, 50)
rank_seq_epa = seq(50,400, by=50)
minimum_frobenius_error_epa = seq(50,400, by=50)

for(ranki in seq(along=rank_seq_epa)){
  
  warm =NULL
  
for( i in seq(along=lambda_values_epa)){
  
  fits_epa = softImpute(epa2,lambda=lambda_values_epa[i],rank=rank_seq_epa[ranki], maxit=1000, type='svd')
  
 
  W_epa = fits_epa$u %*% diag(fits_epa$d) %*% t(fits_epa$v)
  
  f_epa = epa-W_epa
  
  frobenius_epa[i]  = sqrt(sum(f_epa^2, na.rm=TRUE))
  
}
  minimum_frobenius_error_epa[ranki] = min(frobenius_epa)
  plot(lambda_values_epa, frobenius_epa, main=rank_seq_epa[ranki])

}
```

## S-PrediXcan work
First, have to normalize the codes using the work we already did:
- Get names of phenotypes in the S-PrediXcan files
- Map them to their shared names with Sider (using the "ukb" (uk biobank phenotype) column)
- Using onlythe shared phenotypes we found, make a mapping of the phenotype columns to the sider phenotypes (spredi2name)  


In [79]:
import re
d2code = dict(tuple(spredi.loc[:,['full_code','description']].values))

sp = pd.read_csv("/project/uml_rachel_melamed/predixcan/spredixcan/spredixcan-Adipose_Subcutaneous.tsv.gz",sep="\t",nrows=20,index_col=0)
#sp = sp.rename(columns=d2code)
orig = sp.columns
sp = sp.rename(columns={re.sub("[^0-9a-zA-Z_]+","_",i):i for i in spredi0['full_code']}) ## make it match "full_code"
sp = sp.rename(columns=d2code)
usecols = orig[sp.columns.isin(set(pt_match['ukb']))]

d1 = {re.sub("[^0-9a-zA-Z_]+","_",i):i for i in spredi0['full_code']}
d2 = {k:d2code[v] for k,v in d1.items()}
spredi2name = {k:d2[k] for k in usecols}


n2spr = {v:k for k,v in spredi2name.items()}


# the name matched to sider mapping to multix/spredix
n2full_code = {k:d1[n2spr[k]] for k in n2spr}

Now, load in the individual tissues in order to obtain their *sign* of S-PrediXcan effect of genetics on tissue expression on phenotype. 

In [74]:
def get_tissue_expr(i):
    tname = os.path.basename(i).replace(".tsv.gz","").replace("spredixcan-","")
    tt = pd.read_csv(i,sep="\t",index_col=0, usecols=['gene_name'] + [str(k) for k in spredi2name.keys()])
    tt = tt.rename(columns=spredi2name)    
    tt = tt.loc[pd.isnull(tt).sum(axis=1)<.8*tt.shape[1],:]

    return tt

In [97]:
tot = pd.DataFrame()
spredixcan 
for i in glob.glob("/project/uml_rachel_melamed/predixcan/spredixcan/*tsv.gz"):
    if 'smultixcan' in i:
        continue
    tname = os.path.basename(i).replace(".tsv.gz","").replace("spredixcan-","")
    blood = get_tissue_expr(i)
    blood = blood.groupby(blood.index).mean() 
    blood = pd.DataFrame(blood > 0,dtype=int)*2 - 1M
    if tot.shape[0]==0:
        tot = blood
    else:
        ix_inters = blood.index.intersection(tot.index)
        #tot +=%%! blood.loc[ix_inters,]
        tot.loc[ix_inters,:] += blood.loc[ix_inters,]
        ix_diff = blood.index.difference(tot.index)
        tot = pd.concat((tot, blood.loc[ix_diff,:]),axis=0)
tot.columns = [i.strip() for i in tot.columns]

Then load in the MultiXcan results-- again convert to our phenotypes 

In [98]:
engs = pd.read_table("input_data/ensg2gene_name.txt",sep="\t",index_col=0)
engs['en2'] = [i.split(".")[0] for i in engs.index]
multix = pd.read_table(spredixcan + 'smultixcan-mashr-pvalues.tsv.gz')
multix['name'] = engs.set_index('en2').loc[multix['gene_name'],'gene_name'].values

multix2 = pd.concat({i:multix[n2full_code[i]] for i in tot.columns},axis=1)
multix2.index = multix['gene_name']
multix2 = multix2.loc[tot.index,:]

del multix

Finally, convert the p-values to their quantiles (z-statistics) and use the sign calculated above to give them a positive/ negative sign

In [107]:
from scipy.stats import norm
mzdfp = {dis:(np.abs(norm.ppf(np.clip(multix2.loc[:,dis],0,1-np.exp(-10))))*tot[dis].values)
                  for dis in multix2.columns}
mzdfp = pd.DataFrame(mzdfp, index=multix2.index)

mzdfp = mzdfp.loc[mzdfp.isnull().sum(axis=1) == 0,:]

/share/pkg/python3/3.6_intel_pu3_2018.3.039/intelpython3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2017: RuntimeWarning: invalid value encountered in greater
  cond1 = (0 < q) & (q < 1)
/share/pkg/python3/3.6_intel_pu3_2018.3.039/intelpython3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:2017: RuntimeWarning: invalid value encountered in less
  cond1 = (0 < q) & (q < 1)


This is 20000 genes but to reduce the problem, we take only the top 10000~ genes with the highest variance.

In [125]:
mfil = mzdfp.loc[mzdfp.var(axis=1) > 420,:]
mfil.to_csv("data_for_regression/multixcan_res_10027.txt",sep="\t")

In [ ]:
pt

In [126]:
ptstack.head()

pubchem_cid,85,119,137,143,158,159,160,175,187,191,...,53477714,54677977,54681041,54682541,54687131,56603655,56842239,70683024,70695640,71306834
ukb,,,,,,,,,,,,,,,,,,,,,
#Arthrosis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
AV-block,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Actinic keratosis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Anxiety disorders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Atrophic disorders of skin,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
t = pd.read_csv("../../epa_sider_cancer/multixcan_res_10027.txt",sep="\t",index_col=0)

In [124]:
mfil.head()

,Mood swings,Irritability,"Cancer code, self-reported: lung cancer","Cancer code, self-reported: non-hodgkins lymphoma","Cancer code, self-reported: malignant melanoma","Cancer code, self-reported: basal cell carcinoma","Cancer code, self-reported: squamous cell carcinoma","Non-cancer illness code, self-reported: hypertension","Non-cancer illness code, self-reported: peripheral vascular disease","Non-cancer illness code, self-reported: angina",...,Diagnoses - main ICD10: R35 Polyuria,Diagnoses - main ICD10: R42 Dizziness and giddiness,Diagnoses - main ICD10: R51 Headache,"Diagnoses - main ICD10: R52 Pain, not elsewhere classified",Diagnoses - main ICD10: R55 Syncope and collapse,Rheumatoid Arthritis,Diseases of the nervous system,Diseases of the genitourinary system,Diseases of the respiratory system,Schizophrenia
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000457,9.478577,2.099586,5.225691,14.343574,19.214453,-35.752534,3.105903,-19.760873,69.825017,72.099688,...,25.212429,-68.259117,5.373495,-8.028179,-30.349523,0.379065,-32.763998,-3.779217,-22.467705,87.156460
ENSG00000000938,-23.906707,-9.920647,-5.002031,15.288431,1.879648,-34.363908,-44.153022,-83.546183,22.705351,-54.502511,...,-21.492087,26.170673,6.637328,-5.239296,-40.420952,-49.574653,-13.781242,25.492967,18.145799,40.589279
ENSG00000001460,0.595514,-6.419101,-10.434091,-42.859342,-13.842170,19.588182,63.310981,16.226332,0.545078,14.733176,...,0.064792,-15.386067,11.131495,-16.272299,11.560008,-22.225548,-1.947553,3.135598,4.043798,48.696662
ENSG00000001617,-79.515698,-29.750328,2.401402,-11.592920,24.176375,17.216005,12.751756,-11.378498,-8.188700,-7.395245,...,-13.413978,-14.704419,18.676515,-0.487851,-18.883770,4.996079,-51.884142,-55.078373,-42.383016,14.086786
ENSG00000001629,11.493049,7.353918,0.000000,-21.616664,-2.350974,-42.703503,-4.322356,0.000000,0.909765,-37.919704,...,-44.635659,50.294047,18.847470,0.000000,-7.663519,39.584006,25.115217,9.093897,2.020064,-5.490537


In [78]:
spredi0 = spredi

In [85]:
sn = json.load(open('/home/rm97l/wrk/drug_integration/work/21.05.01_epa/spredi2name_dec21.json'))

In [86]:
len(sn)

197

In [80]:
len(spredi2name)

197

In [87]:
spredi2name

{'1920_Mood_swings': 'Mood swings',
 '1940_Irritability': 'Irritability',
 '20001_1001_Cancer_code_selfreported_lung_cancer': 'Cancer code, self-reported: lung cancer',
 '20001_1053_Cancer_code_selfreported_nonhodgkins_lymphoma': 'Cancer code, self-reported: non-hodgkins lymphoma',
 '20001_1059_Cancer_code_selfreported_malignant_melanoma': 'Cancer code, self-reported: malignant melanoma',
 '20001_1061_Cancer_code_selfreported_basal_cell_carcinoma': 'Cancer code, self-reported: basal cell carcinoma',
 '20001_1062_Cancer_code_selfreported_squamous_cell_carcinoma': 'Cancer code, self-reported: squamous cell carcinoma',
 '20002_1065_Noncancer_illness_code_selfreported_hypertension': 'Non-cancer illness code, self-reported: hypertension',
 '20002_1067_Noncancer_illness_code_selfreported_peripheral_vascular_disease': 'Non-cancer illness code, self-reported: peripheral vascular disease',
 '20002_1074_Noncancer_illness_code_selfreported_angina': 'Non-cancer illness code, self-reported: angina'

In [91]:
set(spredi2name.keys())- set(sn.keys()) 

set()

In [ ]:
tot = pd.DataFrame()
for i in glob.glob("/project/uml_rachel_melamed/predixcan/ukb_spredixcan_pertissue/*txt"):
    blood = pd.read_table(i,sep=",",index_col=0)
    blood = blood.groupby(blood.index).mean() ## some duplicate genes
    blood = pd.DataFrame(blood > 0,dtype=int)*2 - 1
    if tot.shape[0]==0:
        tot = blood
    else:
        ix_inters = blood.index.intersection(tot.index)
        #tot +=%%! blood.loc[ix_inters,]
        tot.loc[ix_inters,:] += blood.loc[ix_inters,]
        ix_diff = blood.index.difference(tot.index)
        tot = pd.concat((tot, blood.loc[ix_diff,:]),axis=0)
tot.columns = [i.strip() for i in tot.columns]

In [ ]:
for i in glob.glob("/project/uml_rachel_melamed/predixcan/spredixcan/*tsv.gz"):
    tname = os.path.basename(i).replace(".tsv.gz","").replace("spredixcan-","")
    tt = get_tissue_expr(i)
    savemat("mats/" + tname + '.mat',
        {'rownames':list(tt.index), 'colnames':list(tt.columns), 'dat':tt.values})